In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import scipy.stats as sci
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

***
### Data Import


In [ ]:
# test = pd.read_csv("C:/Users/Avery/Documents/GitHub/Regression_proj/dsProject/test_sample.csv")
train = pd.read_csv("C:/Users/Avery/Documents/GitHub/Regression_proj/dsProject/train_sample.csv")
train.drop(columns='Unnamed: 0',inplace= True)

***
### Misc Functions

In [ ]:
# Cleaning the columns with T/F and making them 1/0
for i in range(1,10):
    train['M'+str(i)] = train['M'+str(i)].replace({'T': 1, 'F': 0})

In [ ]:
# Some code I found that will go through and mark the unique entries in each column, giving the general estimate of what each does
for col, values in train.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

In [ ]:
dtypething = {k:v for k, v in (train.dtypes == 'O').to_dict().items() if v is True}


for k, v in dtypething.items():
    train[k] = train[k].astype(str)

***
### Distribution Graphs


In [ ]:
transactions = train[train['TransactionAmt'] >0 ]['TransactionAmt']

# Dist of TransactionAmt
fig = px.histogram(
    transactions, 
    title= "Distribuiton of Transactions",
    nbins=1000
    ) 

fig.update_layout( height=500, width=1000)
fig.update_xaxes(range = [0,5000])

In [ ]:
transactions = train[train['TransactionAmt'] > 0 ]['TransactionAmt']

# Dist of TransactionAmt
fig = px.box(
    transactions, 
    title= "Distribuiton of Transactions"
    ) 

fig.update_layout( height=500, width=750)
# fig.update_xaxes(range = [0,5000])
fig.show()

In [ ]:
Card_Dist = pd.value_counts(train['card4'],normalize= True, dropna = False)

# Dist of Card Types
fig = px.bar( 
    data_frame = Card_Dist,
    title='Distribution of Card Type',color = Card_Dist.keys(),
    text= Card_Dist.apply(lambda x: f'{x*100:.2f}%'),
    orientation='h'
    )

fig.update_layout(xaxis={'title': 'Card Types'},yaxis={'title': 'Percentage'}, height=500, width=1000)

In [ ]:
dist = pd.value_counts(train['DeviceInfo'],normalize= True, dropna = False)

amount_to_show = 9
new_data = {}

for i, (key, value) in enumerate(dist.to_dict().items()):
    if i < amount_to_show:
        new_data[key] = value
    else:
        new_data['Other'] = new_data.get('Other', 0) + value

fig = px.pie(
    values=list(new_data.values()), 
    names=list(new_data.keys()),
    title= 'Distribution Of Operating System That Made The Transaction'
    )

fig.update_traces(textinfo='percent+label')
fig.show()

***
### Confidence Interval Stuff

In [ ]:
proplist = [1,2,3,5,6,7,8,9]
card_props = []
for i in proplist:
    prop = train['M'+str(i)].mean()
    card_props.append(prop)
total_card = len(card_props)

for i in card_props:
    se = np.sqrt(i*(1-i)/total_card)
    me = sci.t.ppf(0.975, total_card-1)*se
    ci = (i-me, i+me)
    print('=======================================')
    print('Sample Proportion:', i)
    print('Confidence Interval (95%):', ci)


***
### Some Random PCA stuff

In [ ]:
supertestset = train.copy()

supertestset = pd.get_dummies(supertestset,prefix_sep = '_')

In [ ]:
# # Some PCA testing



supertestset2 = supertestset.copy()
supertestset2.dropna(axis= 1,inplace= True)

# data_scalar = StandardScaler()
# data_scalar.fit(supertestset2)
# scaled_data_frame = data_scalar.transform(supertestset2)

# pca = PCA(n_components=2)
# pca.fit(scaled_data_frame)
# x_pca = pca.transform(scaled_data_frame)


# plt.scatter(x_pca[:,0],x_pca[:,1],c=supertestset2['isFraud'])

***
### Random Forest Stuff

Our top 10 vars!


In [ ]:
X = supertestset2.drop(['isFraud'],axis = 1)
Y = supertestset2['isFraud']
Y = LabelEncoder().fit_transform(Y)
X2 = StandardScaler().fit_transform(X)
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X2,Y,test_size = 0.3, random_state = 101)

trainedForest = RandomForestClassifier(n_estimators = 700).fit(X_Train,Y_Train)
prediciton = trainedForest.predict(X_Test)

In [ ]:
feat_importances = pd.Series(trainedForest.feature_importances_, index = X.columns)
feat_importances.nlargest(7).plot(kind='barh')

In [ ]:
importance = feat_importances.nlargest(10).keys().to_list()

X_Reduced = X[importance]
X_Reduced = StandardScaler().fit_transform(X_Reduced)
X_Train2, X_Test2, Y_Train2, Y_Test2 = train_test_split(X_Reduced, Y, test_size = 0.30, random_state = 101)


trainedforest = RandomForestClassifier(n_estimators=700).fit(X_Train2,Y_Train2)
predictionforest = trainedforest.predict(X_Test2)

VERY SLIGHT DROP IN PREFORMANCE

In [ ]:
print('================================================================')
print('All Features')
print(classification_report(Y_Test,prediciton))
print('================================================================')
print('Top 10 Features')
print(classification_report(Y_Test2,predictionforest))
print('================================================================')

***